# Experiment Tracking with MLflow - Bridge Failure Prediction

In [1]:
import mlflow
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import joblib

## Load Data

In [2]:
df = pd.read_csv('../data/processed/features.csv')
X = df.drop(['failure_within_1yr', 'structure_id'], axis=1)
y = df['failure_within_1yr']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Run MLflow Experiment

In [3]:
mlflow.set_experiment("bridge_failure_prediction")
with mlflow.start_run():
    params = {"n_estimators": 100, "max_depth": 8, "random_state": 42}
    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)
    y_proba = clf.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    mlflow.log_params(params)
    mlflow.log_metric("roc_auc", auc)
    joblib.dump(clf, "model.joblib")
    mlflow.log_artifact("model.joblib")
    print("AUC logged to MLflow:", auc)

## View Results
Run `mlflow ui` in your terminal and navigate to [http://localhost:5000](http://localhost:5000) to view experiment runs and artifacts.